# import Statements

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint


In [2]:
df = pd.read_csv("intrusion_detection/data.csv")
# df = df.rename(columns={"left": "label"})


In [3]:
df.head()

,duration,service,src_bytes,dst_bytes,hot,num_failed_logins,num_compromised,num_root,num_file_creations,num_access_files,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,xAttack
0,0,25,193,441,0,0,0,0,0,0,...,255,1.00,0.00,0.07,0.04,0.00,0.04,0.0,0.0,normal
1,0,38,0,0,0,0,0,0,0,0,...,1,0.00,0.07,0.00,0.00,0.00,0.00,1.0,1.0,dos
2,0,25,167,9724,0,0,0,0,0,0,...,255,1.00,0.00,0.03,0.06,0.00,0.00,0.0,0.0,normal
3,0,20,1339,0,0,0,0,0,0,0,...,31,0.23,0.04,0.23,0.00,0.02,0.00,0.0,0.0,normal
4,0,37,0,0,0,0,0,0,0,0,...,25,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.0,dos


In [4]:
len(df)
# df.dtypes

24998

In [5]:
# df.info()

# Training and Validation data split

In [6]:
def train_validate_split(df,test_size):
    if isinstance(test_size,float):
        test_size = round(test_size*len(df))
        
    indices=df.index.tolist()
    validate_indices = random.sample(population=indices,k=test_size)
    
    validate_df=df.loc[validate_indices]
    train_df=df.drop(validate_indices)
    return train_df,validate_df

In [7]:
# random.seed(0)
# train_df,validate_df=train_validate_split(df,test_size=0.2)

In [8]:
data = df.values
data2= df.values
data=data[:,:-1]
label=df.values
label=label[:,-1]
# validate_df.head()
name_list=list(df)

# print(data.shape,label.shape)

mean_list=[]
new_values=data.transpose()
# print(new_values.shape,data.shape)
# for x in new_values:
#     mn=np.mean(x,axis=0)
#     mean_list.append(mn)

# print(mean_list)
X = df.loc[:,'duration':'dst_host_srv_rerror_rate']

# for i in range(len(data)):
X.loc[:,'duration':'dst_host_srv_rerror_rate']=(X.loc[:,'duration':'dst_host_srv_rerror_rate'] - X.loc[:,'duration':'dst_host_srv_rerror_rate'].mean())/X.loc[:,'duration':'dst_host_srv_rerror_rate'].std()
    
# print(X.shape)
# Q=X.transpose()
# print(Q.shape)
# Q=np.matmul(X,Q)
# print(Q.shape)
covarience=np.cov(np.array(X.T))
# print(covarience.shape)

eigen_values,eigen_vectors=np.linalg.eig(covarience)
# print(eigen_values)

max_val=[]
for i in range(len(eigen_values)):
    temp=eigen_values[i]/sum(eigen_values)
    max_val.append((temp,i))
    
max_val=sorted(max_val)
max_val.reverse()
# print(max_val)
newdata_list=[]
temp=0

for x in max_val:
    if temp >= (.9):
        break
    temp+=x[0]
    newdata_list.append(x[1])

# print(type(data),type(new_data))
new_data=eigen_vectors[:,newdata_list]    
# print()
new_data=np.dot(X,new_data)

In [9]:
def distances(test,centroids):
#     distance=0
    distances=[]
    for i in range(5):    
        centroid=centroids[i+1]
        distance=0
        for j in range(14):
            distance=distance+pow((test[j]-centroid[j]),2)
        distances.append(math.sqrt(distance))
#     print(distances)
    return distances

In [10]:
np.random.seed(200)
k = 5

centroids = {}
for i in range(k):
    centroids[i+1]=new_data[random.randint(0,12000)]
# print(centroids)

data_dic={}
for i in range(len(new_data)):
    distance=distances(new_data[i],centroids)
    data_dic[i]=distance.index(min(distance))+1
    
# print(data_dic)
# count=[0,0,0,0,0]
list1=[]
list2=[]
list3=[]
list4=[]
list5=[]
for i in data_dic:
    x=data_dic[i]
    if x==1:
        list1.append(new_data[i])
    elif x==2:
        list2.append(new_data[i])
    elif x==3:
        list3.append(new_data[i])
    elif x==4:
        list4.append(new_data[i])
    elif x==5:
        list5.append(new_data[i])
        
#     count[data_dic[i]-1]+=1
    
# print(count)
print(len(list1))
print(len(list2))
print(len(list3))
print(len(list4))
print(len(list5))

mn=[0,0,0,0,0,0,0,0,0,0,0,0,0,0]

for i in range(25):
    new_centroids={}
    
#     print(len(mn))
    mn=[float(sum(l))/len(l) for l in zip(*list1)]
    new_centroids[1]=mn

    mn=[float(sum(l))/len(l) for l in zip(*list2)]
    new_centroids[2]=mn

    mn=[float(sum(l))/len(l) for l in zip(*list3)]
    new_centroids[3]=mn

    mn=[float(sum(l))/len(l) for l in zip(*list4)]
    new_centroids[4]=mn

    mn=[float(sum(l))/len(l) for l in zip(*list5)]
    new_centroids[5]=mn

    data_dic={}
    for i in range(len(new_data)):
        distance=distances(new_data[i],new_centroids)
        data_dic[i]=distance.index(min(distance))+1

#     print(data_dic)
    new_centroids={}
#     count=[0,0,0,0,0]
    list1=[]
    list2=[]
    list3=[]
    list4=[]
    list5=[]
    
    final_list1=[]
    final_list2=[]
    final_list3=[]
    final_list4=[]
    final_list5=[]

    for i in data_dic:
        x=data_dic[i]
        if x==1:
            list1.append(new_data[i])
            final_list1.append(data2[i])
        elif x==2:
            list2.append(new_data[i])
            final_list2.append(data2[i])
        elif x==3:
            list3.append(new_data[i])
            final_list3.append(data2[i])
        elif x==4:
            list4.append(new_data[i])
            final_list4.append(data2[i])
        elif x==5:
            list5.append(new_data[i])
            final_list5.append(data2[i])
    
    print()
    print(len(list1),len(final_list1))
    print(len(list2),len(final_list2))
    print(len(list3),len(final_list3))
    print(len(list4),len(final_list4))
    print(len(list5),len(final_list5))
    
# a = np.array(a)
y=np.array(final_list1)
uniq,count=np.unique(y[:,-1],return_counts=True)
catagory1=uniq[np.argmax(count)]
accuracy1=max(count)/np.sum(count)
print(catagory1,accuracy1)

y=np.array(final_list2)
uniq,count=np.unique(y[:,-1],return_counts=True)
catagory2=uniq[np.argmax(count)]
accuracy2=max(count)/np.sum(count)
print(catagory2,accuracy2)

y=np.array(final_list3)
uniq,count=np.unique(y[:,-1],return_counts=True)
catagory3=uniq[np.argmax(count)]
accuracy3=max(count)/np.sum(count)
print(catagory3,accuracy3)

y=np.array(final_list4)
uniq,count=np.unique(y[:,-1],return_counts=True)
catagory4=uniq[np.argmax(count)]
accuracy4=max(count)/np.sum(count)
print(catagory4,accuracy4)

y=np.array(final_list5)
uniq,count=np.unique(y[:,-1],return_counts=True)
catagory5=uniq[np.argmax(count)]
accuracy5=max(count)/np.sum(count)
print(catagory5,accuracy5)

final_acc=(accuracy1+accuracy2+accuracy3+accuracy4+accuracy5)/5

print("final_acc  : ",final_acc )

13121
2740
2200
4026
2911

13686 13686
1625 1625
2876 2876
5204 5204
1607 1607

13282 13282
1521 1521
2942 2942
6633 6633
620 620

12889 12889
1513 1513
2934 2934
6840 6840
822 822

12764 12764
1476 1476
2934 2934
6843 6843
981 981

12676 12676
1445 1445
2934 2934
6843 6843
1100 1100

12608 12608
1424 1424
2934 2934
6843 6843
1189 1189

12554 12554
1408 1408
2934 2934
6842 6842
1260 1260

12508 12508
1396 1396
2934 2934
6842 6842
1318 1318

12464 12464
1389 1389
2934 2934
6841 6841
1370 1370

12381 12381
1379 1379
2934 2934
6841 6841
1463 1463

12233 12233
1376 1376
2934 2934
6841 6841
1614 1614

12099 12099
1375 1375
2934 2934
6841 6841
1749 1749

12048 12048
1374 1374
2933 2933
6841 6841
1802 1802

12037 12037
1374 1374
2933 2933
6841 6841
1813 1813

12036 12036
1374 1374
2933 2933
6841 6841
1814 1814


KeyboardInterrupt: 